In [1]:
# Import the CockpitATS client and required modules
import sys
import os
from datetime import datetime
from clients.cockpit import CockpitATSClient

print("✅ Successfully imported CockpitATSClient")
print(f"Current time: {datetime.now()}")

ModuleNotFoundError: No module named 'clients'

In [ ]:
# For schedule data (employee data and employee schedule)

# File: planbition_client.py

import requests
import os
import logging
from datetime import datetime, timedelta
from typing import Optional, Dict, Any, List

# Set up logging
logger = logging.getLogger(__name__)

class PlanbitionClient:
    """
    Client for the Planbition REST API.
    
    Handles JWT token authentication and provides methods for
    retrieving schedule data as required by the GetScheduleTool.
    """
    
    def __init__(self):
        """Initializes the client by loading credentials from environment variables."""
        self.base_url = os.environ.get("PLANBITION_BASE_URL")
        self.api_key = os.environ.get("PLANBITION_KEY")
        self.username = os.environ.get("PLANBITION_USERNAME")
        self.password = os.environ.get("PLANBITION_PASSWORD")
        
        if not all([self.base_url, self.api_key, self.username, self.password]):
            logger.error("PLANBITION_BASE_URL, KEY, USERNAME, or PASSWORD not set.")
            raise ValueError("Planbition API credentials are not fully configured.")
            
        self._bearer_token: Optional[str] = None
        self._token_expiry = datetime.now()

    def _get_bearer_token(self) -> str:
        """
        Retrieves a valid 60-minute bearer token, authenticating if necessary.
        
        This method caches the token until it's within a 1-minute
        expiry window.
        """
        # Check if current token is valid (with a 1-minute buffer)
        if self._bearer_token and self._token_expiry > (datetime.now() + timedelta(minutes=1)):
            return self._bearer_token
            
        # Token is invalid or expired, perform authentication
        auth_url = f"{self.base_url}/authenticate/login"
        auth_payload = {
            "Key": self.api_key,
            "UserName": self.username,
            "Password": self.password
        }
        
        try:
            logger.info("Authenticating with Planbition API...")
            response = requests.post(auth_url, json=auth_payload)
            response.raise_for_status()
            
            response_data = response.json()
            if not response_data.get("success") or not response_data.get("token"):
                logger.error(f"Planbition auth failed: {response_data.get('error')}")
                raise Exception(f"Planbition auth failed: {response_data.get('error', 'No token')}")

            self._bearer_token = response_data["token"]
            # Token is valid for 60 minutes
            self._token_expiry = datetime.now() + timedelta(minutes=60)
            logger.info("Successfully retrieved Planbition token.")
            return self._bearer_token
            
        except requests.exceptions.RequestException as e:
            logger.error(f"Error during Planbition authentication: {e}")
            raise

    def _make_request(self, method: str, endpoint: str, params: Optional[Dict[str, Any]] = None) -> Any:
        """Helper function to make authenticated API requests."""
        token = self._get_bearer_token()
        
        url = f"{self.base_url}/api/{endpoint}" # All API endpoints are prefixed with /api/
        headers = {
            "Authorization": f"Bearer {token}", 
            "Content-Type": "application/json",
            "Accept": "text/plain" 
        }
        
        try:
            response = requests.request(method, url, headers=headers, params=params)
            
            # Handle token expiry (403 Forbidden is a common response)
            if response.status_code == 403: 
                logger.warning("Received 403 (Forbidden), retrying with new token.")
                self._bearer_token = None # Force re-authentication
                token = self._get_bearer_token()
                headers["Authorization"] = f"Bearer {token}"
                response = requests.request(method, url, headers=headers, params=params)

            response.raise_for_status()
            return response.json()
            
        except requests.exceptions.RequestException as e:
            logger.error(f"Error calling Planbition API {endpoint}: {e}")
            raise

    def get_employee_schedule(self, employee_number: str, start_date: str, end_date: str) -> List[Dict[str, Any]]:
        """
        Fetches the schedule for a specific employee within a date range.
        
        This uses the ScheduleEmployeeShiftDemand GET endpoint and filters
        by employeeNumber and the shift's start time.
        
        Args:
            employee_number: The unique employee number of the flex worker.
            start_date: The start of the date range (e.g., "YYYY-MM-DD").
            end_date: The end of the date range (e.g., "YYYY-MM-DD").
            
        Returns:
            A list of schedule entries.
        """
        
        endpoint = "ScheduleEmployeeShiftDemand" # [cite: 3243]
        
        # This filter is more robust because 'employeeNumber' is a
        # confirmed field in the GET response for this endpoint.
        filters = [
            f"employeeNumber eq '{employee_number}'",
            f"startTime ge {start_date}", # Filter for shifts starting on or after this date
            f"startTime le {end_date}"  # Filter for shifts starting on or before this date
        ]
        
        params = {
            "filter": " and ".join(filters), # [cite: 1566-1567]
            "PageNumber": 1,                 # [cite: 1513]
            "PageSize": 100                  # [cite: 1517, 1521]
        }
        
        try:
            return self._make_request("GET", endpoint, params=params)
        except Exception as e:
            print(f"Could not retrieve schedule for employee {employee_number}: {e}")
            return []

In [ ]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from planbition import PlanbitionClient# Import the client from Step 1

# Initialize the client once
client = PlanbitionClient()

class ScheduleInput(BaseModel):
    employee_number: str = Field(description="The unique employee number for the flex worker.")
    start_date: str = Field(description="The start date of the period to query, in YYYY-MM-DD format.")
    end_date: str = Field(description="The end date of the period to query, in YYYY-MM-DD format.")

@tool(args_schema=ScheduleInput)
def get_schedule(employee_number: str, start_date: str, end_date: str) -> List[Dict[str, Any]]:
    """
    Retrieves the work schedule for a flex worker from the Planbition API
    for a specified date range. This tool is called by the ActionExecutionAgent
    when a user asks about their schedule.
    """
    try:
        # The client handles all auth and filtering
        schedule_data = client.get_schedule(
            employee_number=employee_number,
            start_date=start_date,
            end_date=end_date
        )
        # The tool returns the raw structured data
        # The AnswerAgent will be responsible for formatting this
        # into a user-friendly response
        return schedule_data
    except Exception as e:
        # The agent's error handling will catch this
        return f"Error retrieving schedule: {e}"

In [ ]:
# Test the PlanbitionClient and get_schedule tool
print("Testing PlanbitionClient and get_schedule tool...")

# Test direct client call
try:
    # Replace with actual employee number and dates for testing
    test_employee = "12345"  # Replace with a real employee number
    test_start = "2024-01-01"
    test_end = "2024-01-07"
    
    print(f"Testing direct client call for employee {test_employee}")
    schedule_data = client.get_employee_schedule(test_employee, test_start, test_end)
    print(f"Direct client result: {len(schedule_data) if isinstance(schedule_data, list) else 'Error'} records")
    
except Exception as e:
    print(f"Direct client test failed: {e}")

# Test the LangChain tool
try:
    print(f"\nTesting LangChain tool for employee {test_employee}")
    tool_result = get_schedule.invoke({
        "employee_number": test_employee,
        "start_date": test_start,
        "end_date": test_end
    })
    print(f"Tool result: {type(tool_result)} - {len(tool_result) if isinstance(tool_result, list) else tool_result}")
    
except Exception as e:
    print(f"Tool test failed: {e}")

print("✅ Testing complete")

Testing PlanbitionClient and get_schedule tool...
Testing direct client call for employee 12345
Direct client result: Error records

Testing LangChain tool for employee 12345
Tool result: <class 'str'> - Error retrieving schedule: 'PlanbitionClient' object has no attribute 'get_schedule'
✅ Testing complete
